Installing Dependencies

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00


HuggingFace Login

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

Loading Model and Tokenizer: LLaMa-2-7b-chat-hf model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Load LlAMa-2 model (on GPU 0)
device_map = {"": 0}

# Load in low precision
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Model
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             quantization_config=bnb_config,
                                             device_map=device_map)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Define LoRA Parameters Configuration

In [ ]:
from peft import LoraConfig

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

Define Training parameters

In [ ]:
from transformers import TrainingArguments

# Set training parameters
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard"
)

Load Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("mlabonne/guanaco-llama2-1k")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Data PreProcessing

In [ ]:
print(dataset['train'])

Dataset({
    features: ['text'],
    num_rows: 1000
})


In [ ]:
# tokenize data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset['train'].map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Train Model

In [ ]:
from trl import SFTTrainer

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_datasets,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=None
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("LlaMA2Chatbot")

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.412500
50,1.654700
75,1.226300
100,1.457900
125,1.189900
150,1.382000
175,1.185600
200,1.485300
225,1.169200
250,1.555500


Hyperparameter Tuning

In [ ]:
# Random Search of hyperparameters.

# First Attempt
# Set training parameters
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=6e-4, # new change
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard"
)
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_datasets,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=None
)
trainer.train()
trainer.model.save_pretrained("LlaMA2Chatbot/fine-tuning1")

# Second Attempt
# Set training parameters
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.003, # new change
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard"
)
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_datasets,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=None
)
trainer.train()
trainer.model.save_pretrained("LlaMA2Chatbot/fine-tuning2")

# Third Attempt
# Set training parameters
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.09, # new change
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard"
)
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_datasets,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=None
)
trainer.train()
trainer.model.save_pretrained("LlaMA2Chatbot/fine-tuning3")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Step,Training Loss
25,1.311800
50,1.555500
75,1.214700
100,1.434300
125,1.184900
150,1.360800
175,1.172500
200,1.457300
225,1.150800
250,1.529100


Step,Training Loss
25,1.411900
50,1.656200
75,1.229600
100,1.451200
125,1.189500
150,1.376700
175,1.183500
200,1.472900
225,1.166600
250,1.542600


Step,Training Loss
25,1.503100
50,1.710200
75,1.231900
100,1.461700
125,1.191400
150,1.380300
175,1.185000
200,1.474400
225,1.168100
250,1.543300


Save Final Model to Hugging Face

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [ ]:
!huggingface-cli repo create chatbot413

git version 2.34.1
git-lfs/3.0.2 (GitHub; linux amd64; go 1.18.1)

You are about to create simoHamlili/chatbot413
Proceed? [Y/n] Y

Your repo now lives at:
  https://huggingface.co/simoHamlili/chatbot413

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/simoHamlili/chatbot413



In [ ]:
trainer.push_to_hub(model_name="chatbot413")

To https://huggingface.co/simoHamlili/results
   a3b52c8..7a1701d  main -> main

   a3b52c8..7a1701d  main -> main

To https://huggingface.co/simoHamlili/results
   7a1701d..3f90ccd  main -> main

   7a1701d..3f90ccd  main -> main



'https://huggingface.co/simoHamlili/results/commit/7a1701d45f6d29d9555184fac25bb1ac9a8dd3b1'

Re-Train Model with Lowest Loss

In [ ]:
from trl import SFTTrainer

training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=6e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_datasets,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=None
)

trainer.train()
trainer.model.save_pretrained("LlaMA2Chatbot/fine-tuning1")

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.310800
50,1.554500
75,1.213900
100,1.435700
125,1.186800
150,1.363900
175,1.173500
200,1.460000
225,1.151800
250,1.530200


Model Qualitative Evaluation Using Test Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("mlabonne/guanaco-llama2")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

In [ ]:
from transformers import pipeline

# Sampling 5 English Prompts for Model Eval
nums = [3, 4, 5, 6, 10]
rows = []
for i in nums:
  rows.append(dataset['test']['text'][i])

# Generating an output to the prompt and comparing it qualitatively with the testing dataset's answer.
for row in rows:
  # row = dataset['test']['text'][10]
  start = row.find('[INST]') + 7
  end = row.find('[/INST]')
  prompt = row[start:end]
  startAns = end + 8
  endAns = row.find('[INST]', startAns)
  answer = row[startAns:endAns]


  pipe = pipeline(task="text-generation",model=model,tokenizer=tokenizer,max_length=200,)
  result = pipe(f'<s>[INST] {prompt} [/INST]')
  print(f"Prompt = {prompt}\n\nThe model's answer = \n{result[0]['generated_text'].split('[/INST]')[1]}\n\nThe true answer from the dataset=\n{answer}\n\n\n\n" + "*" * 30)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Prompt = What do you think about ChatGPT? 

The model's answer = 
 ChatGPT is an AI chatbot created by Meta AI. [1] that can generate human-like text based on user input. It is trained on a large dataset of text from the internet and can generate text in response to prompts.

I think it is a great tool for generating human-like text. It has many applications, such as writing articles, generating product descriptions, and even creating entire stories.

One of the most impressive things about ChatGPT is its ability to generate text that is coherent and grammatically correct. It is also able to understand and respond to complex questions and prompts.

However, there are some limitations to ChatGPT. It is not perfect and may sometimes produce inaccurate or irrelevant responses. It is also not able to generate text that is as creative or imagin

The true answer from the dataset=
As an open source alternative to ChatGPT, I do not have personal opinions. However, I can provide objective infor

Run Machine Learning Specific Prompts for Qualitative Evaluation of the Model

In [ ]:
from transformers import pipeline

# define prompts
prompts = [
    "What is a large language model",
    "Can you explain backpropagation?",
    "Give 2 reasons why stochastic gradient descent is better than normal gradient descent",
    "Tell me about neural network architectures.",
]


# create pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

# Evaluate the chatbot on the provided prompts
for prompt in prompts:
    input = f'<s>[INST] {prompt} [/INST]'
    result = pipe(input)
    response = ((result[0]['generated_text']).split('[/INST]'))[-1]

    print("=" * 50)
    print(f"User: {prompt}")
    print(f"Chatbot: {response}")

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


User: What is a large language model
Chatbot:  A large language model (LLM) is a type of artificial intelligence (AI) model that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding. [1] LLMs are often used for tasks such as language translation, text summarization, and chatbots. [2] LLMs are typically trained using deep learning techniques, such as transformers, and are designed to be able to generate text that is similar to that of a human. [3] LLMs are a relatively new field of AI research, and they have the potential to revolutionize the way we interact with machines. [4] 

A large language model is a type of artificial intelligence model that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding. LLMs are often used for tasks such as
User: Can you explain backpropagation?
Chatbot:  Backpropagation is a method used to train artificial neural networks by minimizing the error betw